# Problem Set 7
## Nora Nickels
### Perspectives of Computational Modeling
### Winter 2018

## Problem 1: Classifer "horse" race 

In [4]:
# Import packages

import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.datasets import make_blobs
from pylab import rcParams
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline
rcParams['figure.figsize'] = 10, 8
sb.set_style('whitegrid')
from sklearn import datasets
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import svm, datasets
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report

In [36]:
# Import auto data

auto = pd.read_csv('auto.csv', na_values = "?")

# Drop missing values

auto = auto.dropna()

auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [37]:
# What's the median of auto dataset?
auto = auto.iloc[:, 0:10]
mpg_median = auto['mpg'].median()
mpg_median

22.75

In [38]:
# Create a binary variable of mpg_high
auto['mpg_high'] = auto['mpg'].apply(lambda x: 1 if x >= mpg_median else 0)
auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,mpg_high
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu,0
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320,0
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite,0
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst,0
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino,0


### 1a) Fit a logistic model of mpg_high using k-fold cross validation

In [39]:
import sklearn

from pandas import Series, DataFrame

from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import LeaveOneOut, KFold
from sklearn import metrics 
from sklearn.metrics import classification_report, mean_squared_error
from pylab import rcParams

In [40]:
# Add constant term

auto['const'] = 1
auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,mpg_high,const
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu,0,1
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320,0,1
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite,0,1
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst,0,1
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino,0,1


In [41]:
# Define X and y variables for logit model.

Xvars = auto[['const', 'cylinders', 'displacement', 'horsepower', 'weight',
              'acceleration', 'year', 'origin']].values
yvals = auto['mpg_high'].values

In [42]:
# Define k-fold cross validation with k = 4 folds

k = 4
kf_log = KFold(n_splits=k, random_state=10, shuffle=True)
kf_log.get_n_splits(Xvars)

4

In [51]:
# Define vectors that are k elements long, of MSE's, and errors.

log_err_0 = np.zeros(k)
log_err_1 = np.zeros(k)
log_MSE_vec = np.zeros(k)

In [52]:
# Test model performance with k-fold cross validation

k_ind = int(0)
for train_index, test_index in kf_log.split(Xvars):
    # print("TRAIN:", train_index, "TEST:", test_index)
    print('For k index=', k_ind)
    X_train, X_test = Xvars[train_index], Xvars[test_index]
    y_train, y_test = yvals[train_index], yvals[test_index]
    LogReg = LogisticRegression()
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    err = y_test != y_pred
    log_MSE_vec[k_ind] = err.mean()
    log_err_0[k_ind] = ((y_test == 0) * err).sum() / (y_pred == 0).sum()
    log_err_1[k_ind] = ((y_test == 1) * err).sum() / (y_pred == 1).sum()
    print(classification_report(y_test, y_pred))
    print(log_err_0[k_ind])
    print(log_err_1[k_ind])
    print(log_MSE_vec[k_ind])
    k_ind += 1

For k index= 0
             precision    recall  f1-score   support

          0       0.93      0.88      0.90        57
          1       0.84      0.90      0.87        41

avg / total       0.89      0.89      0.89        98

0.12962962963
0.0909090909091
0.112244897959
For k index= 1
             precision    recall  f1-score   support

          0       0.85      0.83      0.84        42
          1       0.88      0.89      0.88        56

avg / total       0.87      0.87      0.87        98

0.170731707317
0.105263157895
0.132653061224
For k index= 2
             precision    recall  f1-score   support

          0       0.89      0.89      0.89        47
          1       0.90      0.90      0.90        51

avg / total       0.90      0.90      0.90        98

0.106382978723
0.0980392156863
0.102040816327
For k index= 3
             precision    recall  f1-score   support

          0       0.96      0.86      0.91        50
          1       0.87      0.96      0.91        48

In [54]:
# Take average of MSE and error rates for categories across k = 4 test sets.

print(log_MSE_vec.mean())
print(log_err_0.mean())
print(log_err_1.mean())

0.109693877551
0.140574967806
0.0829868283867


* The MSE of the model, reported as the average MSE across the k = 4 test sets, is 0.109693877551
* The error rate for the category when mpg_high = 0, reported as the average error rate for that category across the k = 4 test sets, is 0.140574967806
* The error rate for the category when mpg_high = 1, reported as the average error rate for that category across the k = 4 test sets, is 0.0829868283867

### 1b) Fit a random forest model

In [43]:
from sklearn.ensemble import RandomForestClassifier

In [44]:
rfc = RandomForestClassifier(n_estimators=20, max_features=2, bootstrap=True, oob_score=True, random_state=25)

rfc.fit(Xvars, yvals)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=2, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=True, random_state=25, verbose=0, warm_start=False)

In [45]:
# Because there's no .oob_prediction_ for RCF, we use the oob_decision_function on the predicted values

rfc_pred = rfc.oob_decision_function_

MSE_rfc = pd.DataFrame({'ypred' : rfc_pred.T[1], 'yvals': yvals})
MSE_rfc['ypred'] = MSE_rfc['ypred'].apply(lambda x: 1 if x >= 0.5 else 0)

In [46]:
from sklearn.metrics import mean_squared_error

# Now get MSE

MSE_rfc_full = mean_squared_error(MSE_rfc['yvals'], MSE_rfc['ypred'])
print(MSE_rfc_full)

0.0790816326531


The MSE of the random forest model is 0.0790816326531.

In [50]:
# Now have to get error rates, or MSE's for each category of MSE high.

MSE_rfc_0 = MSE_rfc[MSE_rfc['ypred'] < 0.5]
MSE_0 = mean_squared_error(MSE_rfc_0['yvals'], MSE_rfc_0['ypred'])
print(MSE_0)


MSE_rfc_1 = MSE_rfc[MSE_rfc['ypred'] >= 0.5]
MSE_1 = mean_squared_error(MSE_rfc_1['yvals'], MSE_rfc_1['ypred'])
print(MSE_1)

0.0725388601036
0.0854271356784


* The error rate when mpg_high = 0 is 0.0725388601036.
* The error rate when mpg_high = 1 is 0.0854271356784.

### 1c) Fit a support vector machines model

In [55]:
from sklearn.svm import SVC

# Define vectors 

svm_MSE_vec = np.zeros(k)
svm_error_0 = np.zeros(k)
svm_error_1 = np.zeros(k)

In [58]:
# Test support vector machines model performance with k-fold cross validation
# Use the svc function and rbf kernel!

k_ind = int(0)
for train_index, test_index in kf_log.split(Xvars):
    # print("TRAIN:", train_index, "TEST:", test_index)
    print('k index=', k_ind)
    X_train, X_test = Xvars[train_index], Xvars[test_index]
    y_train, y_test = yvals[train_index], yvals[test_index]
    svc = SVC(kernel='rbf', gamma = 0.2, C=1)
    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    svm_MSE_vec[k_ind] = ((y_test - y_pred) ** 2).mean()
    svm_error_0[k_ind] = ((y_pred==0)*(np.abs(y_test-y_pred))).sum() / (y_pred==0).sum()
    svm_error_1[k_ind] = ((y_pred==1)*(np.abs(y_test-y_pred))).sum() / (y_pred==1).sum()
    print(svm_MSE_vec[k_ind])
    print(svm_error_0[k_ind])
    print(svm_error_1[k_ind])
    print('\n',classification_report(y_test, y_pred, digits=3))
    k_ind += 1

k index= 0
0.571428571429
0.0
0.577319587629

              precision    recall  f1-score   support

          0      1.000     0.018     0.034        57
          1      0.423     1.000     0.594        41

avg / total      0.758     0.429     0.269        98

k index= 1
0.530612244898
0.553191489362
0.0

              precision    recall  f1-score   support

          0      0.447     1.000     0.618        42
          1      1.000     0.071     0.133        56

avg / total      0.763     0.469     0.341        98

k index= 2
0.510204081633
0.515463917526
0.0

              precision    recall  f1-score   support

          0      0.485     1.000     0.653        47
          1      1.000     0.020     0.038        51

avg / total      0.753     0.490     0.333        98

k index= 3
0.5
0.0
0.505154639175

              precision    recall  f1-score   support

          0      1.000     0.020     0.039        50
          1      0.495     1.000     0.662        48

avg / total      

In [59]:
# Take average of MSE and error rates for categories across k = 4 test sets.

print(svm_MSE_vec.mean())
print(svm_error_0.mean())
print(svm_error_1.mean())

0.52806122449
0.267163851722
0.270618556701


* The MSE of the support vector machinesmodel, reported as the average MSE across the k = 4 test sets, is 0.52806122449
* The error rate for the category when mpg_high = 0, reported as the average error rate for that category across the k = 4 test sets, is 0.267163851722
* The error rate for the category when mpg_high = 1, reported as the average error rate for that category across the k = 4 test sets, is 0.270618556701

### 1d) Which of the above three models is the best predictor of mpg_high, and why?

I believe that the random forest model is the best predictor of mph_high, because it has both the lowest MSE and the overall lowest error rates for each mpg_high category. The logit model has a similar error rate for the mpg_high = 1 category (when compared to the random forest model), but a higher error rate for the mph_high = 0 category, and a slightly higher MSE. The SVM model has a drastically higher MSE and higher error rates for each category as well.